<a href="https://colab.research.google.com/github/snakeway8/Projects/blob/main/task_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
data_leads = pd.read_excel('/content/task.xlsx', sheet_name=0)
data_pipelines = pd.read_excel('/content/task.xlsx', sheet_name=1)
data_states = pd.read_excel('/content/task.xlsx', sheet_name=2)
data_crm_users = pd.read_excel('/content/task.xlsx', sheet_name=3)
data_kpi = pd.read_excel('/content/task.xlsx', sheet_name=4)
data_voip_calls = pd.read_excel('/content/task.xlsx', sheet_name=5)
currency = pd.read_excel('/content/currency.xlsx')


###Cleaning data

In [5]:
data_leads.head()

,id,responsible_user_id,created_at,updated_at,is_deleted,closed_at,status_id,sale,pipeline.id
0,17710637.0,2218393.0,1.551039e+09,1.551088e+09,0.0,0.0,18563617.0,0.0,1008472.0
1,17639523.0,2218393.0,1.550771e+09,1.551102e+09,0.0,0.0,18563617.0,0.0,1008472.0
2,17642291.0,2218393.0,1.550836e+09,1.551102e+09,0.0,0.0,18563617.0,0.0,1008472.0
3,16771999.0,2218393.0,1.549447e+09,1.551102e+09,0.0,0.0,18563617.0,0.0,1008472.0
4,17980973.0,2218393.0,1.551174e+09,1.551179e+09,0.0,0.0,18563617.0,0.0,1008472.0


In [6]:
data_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509 entries, 0 to 508
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   507 non-null    float64
 1   responsible_user_id  507 non-null    float64
 2   created_at           507 non-null    float64
 3   updated_at           507 non-null    float64
 4   is_deleted           507 non-null    float64
 5   closed_at            507 non-null    float64
 6   status_id            507 non-null    float64
 7   sale                 507 non-null    float64
 8   pipeline.id          507 non-null    float64
dtypes: float64(9)
memory usage: 35.9 KB


In [7]:
#change data types
data_leads['created_at'] = pd.to_datetime(data_leads['created_at'], unit='s')
data_leads['updated_at'] = pd.to_datetime(data_leads['updated_at'], unit='s')
data_leads['closed_at'] = data_leads['closed_at'].replace(0, pd.NaT)
data_leads['closed_at'] = pd.to_datetime(data_leads['closed_at'], unit='s')



In [8]:
#check null values
columns_to_check = ['id', 'responsible_user_id', 'is_deleted', 'status_id', 'pipeline.id']
na_mask = data_leads[columns_to_check].isna()

rows_with_nan = data_leads[na_mask.any(axis=1)]
rows_with_nan

,id,responsible_user_id,created_at,updated_at,is_deleted,closed_at,status_id,sale,pipeline.id
176,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaN,NaN
177,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaN,NaN


In [9]:
# delete null values and change data types
data_leads = data_leads.dropna(subset=columns_to_check)

data_leads[columns_to_check] = data_leads[columns_to_check].astype('int')


In [10]:
data_leads.dtypes

,0
id,int64
responsible_user_id,int64
created_at,datetime64[ns]
updated_at,datetime64[ns]
is_deleted,int64
closed_at,datetime64[ns]
status_id,int64
sale,float64
pipeline.id,int64


In [11]:
#check duplicates
data_leads.duplicated().sum()

0

In [12]:
#change column names
data_leads.rename(columns={'login':'employeeEmail'}, inplace=True)
data_crm_users.rename(columns={'login':'employeeEmail'}, inplace=True)
data_crm_users.rename(columns={'id':'responsible_user_id'}, inplace=True)
data_pipelines.rename(columns={'id':'pipeline.id'}, inplace=True)
data_kpi.rename(columns={'login':'employeeEmail'}, inplace=True)

### Joining tables

In [13]:
data_leads = data_leads.merge(data_crm_users[['responsible_user_id','name','employeeEmail']],\
                              how='left',on='responsible_user_id')

In [14]:
data_leads.rename(columns={'name':'user_name'},inplace=True)

In [15]:
data_leads = data_leads.merge(data_states[['pipeline.id','state.id']], how='left',on='pipeline.id' )

In [16]:
data_leads=data_leads.merge(data_pipelines[['pipeline.id','name']],how='left')

In [17]:
data_leads.rename(columns={'name':'pipeline_name'},inplace=True)

### Calculation:
total leads,total sales,succesful events,succesful_personal

In [18]:
data_leads['date']=data_leads['created_at'].dt.date

In [19]:
data_leads_group = data_leads.groupby(['date','employeeEmail','user_name'])['id'].nunique().reset_index()
data_leads_group = data_leads_group.rename(columns={'id': 'fact_leads'})


In [20]:
sale_group = data_leads[(pd.notnull(data_leads['closed_at'])) & (data_leads['state.id']== 142)].\
              groupby(['date','employeeEmail','user_name'])['sale'].sum().reset_index()

In [21]:
currency['date']=pd.to_datetime(currency['date'])
sale_group['date']=pd.to_datetime(sale_group['date'])

In [22]:
sale_group = sale_group.merge(currency, how='inner')

In [23]:
sale_group['total_sales_usd'] = round(sale_group['sale']/sale_group['rate'],2)

In [24]:
sale_group = sale_group[['date','employeeEmail','total_sales_usd']]

In [25]:
sale_group.head()

,date,employeeEmail,total_sales_usd
0,2018-03-19,vasya@gmail.ua,0.00
1,2018-03-27,vasya@gmail.ua,26515.15
2,2018-03-29,vasya@gmail.ua,8662.90
3,2018-05-31,vasya@gmail.ua,0.00
4,2018-07-04,vasya@gmail.ua,10137.53


In [26]:
events_data= data_leads[(pd.notnull(data_leads['closed_at'])) &\
                        (data_leads['pipeline_name'] == 'event')&\
                         (data_leads['state.id']== 142)]

In [27]:
personal_data= data_leads[(pd.notnull(data_leads['closed_at'])) &\
                        (data_leads['pipeline_name'] == 'personal')&\
                         (data_leads['state.id']== 142)]

In [28]:
events_data_group = events_data.groupby(['date','employeeEmail','user_name'])['pipeline.id'].count().reset_index()
events_data_group = events_data_group.rename(columns={'pipeline.id': 'succesful_events'})

In [29]:
personal_data_group = personal_data.groupby(['date','employeeEmail','user_name'])['pipeline.id'].count().reset_index()
personal_data_group = personal_data_group.rename(columns={'pipeline.id': 'succesful_personal'})

In [30]:
merged_data_leads = events_data_group.merge(personal_data_group,
                       on=['employeeEmail', 'user_name','date'],
                       how='outer')


In [31]:
merged_data_leads = merged_data_leads.merge(data_leads_group,on=['employeeEmail', 'user_name','date'],
                       how='outer')

In [32]:
merged_data_leads['date'] = pd.to_datetime(merged_data_leads['date'])

In [33]:
merged_data_leads = merged_data_leads.merge(sale_group,on=['employeeEmail','date'],
                       how='outer')

In [34]:
merged_data_leads = merged_data_leads.fillna(0)

In [35]:
merged_data_leads.head()

,date,employeeEmail,user_name,succesful_events,succesful_personal,fact_leads,total_sales_usd
0,2018-03-19,vasya@gmail.ua,Вася,1.0,0.0,1,0.00
1,2018-03-27,vasya@gmail.ua,Вася,1.0,0.0,1,26515.15
2,2018-03-29,vasya@gmail.ua,Вася,1.0,0.0,1,8662.90
3,2018-05-31,vasya@gmail.ua,Вася,1.0,0.0,2,0.00
4,2018-07-04,vasya@gmail.ua,Вася,1.0,0.0,1,10137.53


In [36]:
merged_data_leads_group = merged_data_leads.groupby(['user_name','employeeEmail'])\
                        [['succesful_events',	'succesful_personal',\
                         'fact_leads',	'total_sales_usd']].sum().reset_index()

In [37]:
merged_data_leads_group

,user_name,employeeEmail,succesful_events,succesful_personal,fact_leads,total_sales_usd
0,Аня,anya@gmail.ua,12.0,0.0,72,5777.22
1,Вася,vasya@gmail.ua,5.0,135.0,359,45772.43
2,Максим,maksim@gmail.ua,0.0,0.0,40,0.00
3,Олеся,olesya@gmail.ua,2.0,15.0,36,1281.19


### Cleaning data

In [38]:
data_voip_calls.head()

,internalNumber,startTime,callID,waitsec,billsec,disposition,isNewCall,employeeEmail
0,906,1551885929,1034755903,32,0,CANCEL,0,anya@gmail.ua
1,906,1551885098,1034716873,15,0,CANCEL,0,anya@gmail.ua
2,906,1551884595,1034690958,6,0,CANCEL,0,anya@gmail.ua
3,902,1551882865,1034569923,45,0,BUSY,0,olesya@gmail.ua
4,902,1551882827,1034566968,33,0,CANCEL,0,olesya@gmail.ua


In [39]:
data_voip_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   internalNumber  503 non-null    int64 
 1   startTime       503 non-null    int64 
 2   callID          503 non-null    int64 
 3   waitsec         503 non-null    int64 
 4   billsec         503 non-null    int64 
 5   disposition     503 non-null    object
 6   isNewCall       503 non-null    int64 
 7   employeeEmail   503 non-null    object
dtypes: int64(6), object(2)
memory usage: 31.6+ KB


In [40]:
#change data types
data_voip_calls['startTime'] = pd.to_datetime(data_voip_calls['startTime'], unit='s')

In [41]:
data_voip_calls.dtypes

,0
internalNumber,int64
startTime,datetime64[ns]
callID,int64
waitsec,int64
billsec,int64
disposition,object
isNewCall,int64
employeeEmail,object


In [42]:
#check null values
data_voip_calls.isnull().sum()

,0
internalNumber,0
startTime,0
callID,0
waitsec,0
billsec,0
disposition,0
isNewCall,0
employeeEmail,0


In [43]:
#create new column date
data_voip_calls['date']=data_voip_calls['startTime'].dt.date

In [44]:
#check duplicates
data_voip_calls.duplicated().sum()

0

###Calculation:
 total minutes, succesful minutes

In [45]:
data_voip_calls['total_sec']=data_voip_calls['waitsec']+data_voip_calls['billsec']

In [46]:
group_data_voip_calls = data_voip_calls.groupby(['date','employeeEmail'])\
                                        ['total_sec'].sum().reset_index()

In [47]:
group_data_voip_calls_succesful = data_voip_calls[data_voip_calls['disposition']=='ANSWER'].\
                                  groupby(['date','employeeEmail'])['total_sec'].sum().reset_index()

In [48]:
group_data_voip_calls['fact_min'] = round(group_data_voip_calls['total_sec']/60,0)

In [49]:
group_data_voip_calls_succesful['succesful_min']= round(group_data_voip_calls_succesful['total_sec']/60,0)

In [50]:
merged_data_calls = group_data_voip_calls[['date','employeeEmail','fact_min']].\
              merge(group_data_voip_calls_succesful[['date','employeeEmail','succesful_min']],\
              how='left',on=['date','employeeEmail'])

In [51]:
merged_data_calls.head()

,date,employeeEmail,fact_min,succesful_min
0,2019-02-25,olesya@gmail.ua,67.0,50.0
1,2019-02-25,vasya@gmail.ua,8.0,8.0
2,2019-02-26,anya@gmail.ua,1.0,1.0
3,2019-02-26,maksim@gmail.ua,1.0,1.0
4,2019-02-26,vasya@gmail.ua,2.0,2.0


In [52]:
merged_data_calls_group = merged_data_calls.groupby('employeeEmail')\
                         [['fact_min','succesful_min']].sum().reset_index()
merged_data_calls_group

,employeeEmail,fact_min,succesful_min
0,anya@gmail.ua,47.0,43.0
1,maksim@gmail.ua,153.0,141.0
2,olesya@gmail.ua,281.0,229.0
3,vasya@gmail.ua,25.0,24.0


### Calculation
kpi minutes, kpi leads

In [53]:
data_kpi

,employeeEmail,calls_minutes,leads_nmbr
0,vasya@gmail.ua,30,10
1,olesya@gmail.ua,60,10
2,anya@gmail.ua,30,20
3,maksim@gmail.ua,60,20


In [54]:
merged_data_calls = merged_data_calls.merge(data_crm_users[['name','employeeEmail']], how='left')

In [55]:
merged_data_calls = merged_data_calls.merge(data_kpi[['calls_minutes','employeeEmail']],how = 'left', on='employeeEmail')

In [56]:
merged_data_calls.rename(columns = {'name':'user_name','calls_minutes':'kpi_minutes'},inplace= True)

In [57]:
merged_data_calls['date']=pd.to_datetime(merged_data_calls['date'])

In [58]:
merged_data_leads['week_day'] = merged_data_leads['date'].dt.weekday

In [59]:
merged_data_leads = merged_data_leads.merge(data_kpi[['leads_nmbr','employeeEmail']],how = 'left', on='employeeEmail')

In [60]:
merged_data_leads.rename(columns={'leads_nmbr':'kpi_leads'}, inplace = True)

In [61]:
# KPIs are only issued on working days
merged_data_leads['kpi_leads'] = np.where(merged_data_leads['week_day']<=4,merged_data_leads['kpi_leads'],0)

In [62]:
full_data = merged_data_leads.merge(merged_data_calls, how='outer', on =['date','employeeEmail','user_name'])

In [63]:
full_data = full_data.fillna(0)

In [64]:
full_data = full_data.sort_values(by='date')

In [65]:
result_data = full_data.groupby('user_name')[['kpi_minutes','fact_min','succesful_min','total_sales_usd','kpi_leads',\
                                 'fact_leads','succesful_events','succesful_personal']].sum().reset_index()

In [66]:
result_data

,user_name,kpi_minutes,fact_min,succesful_min,total_sales_usd,kpi_leads,fact_leads,succesful_events,succesful_personal
0,Аня,150.0,47.0,43.0,5777.22,300.0,72.0,12.0,0.0
1,Вася,120.0,25.0,24.0,45772.43,330.0,359.0,5.0,135.0
2,Максим,360.0,153.0,141.0,0.00,220.0,40.0,0.0,0.0
3,Олеся,420.0,281.0,229.0,1281.19,110.0,36.0,2.0,15.0


In [67]:
file_name = "full_data.xlsx"
full_data.to_excel(file_name, index=False)


from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
file_name = "result_data.xlsx"
result_data.to_excel(file_name, index=False)


from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>